In [ ]:
# Imports gerais do projeto
import os
import glob
import numpy as np
import pandas as pd
import scipy.fft
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Descompactar o dataset
zip_name = "archive.zip"
data_dir = "seed_pqd"

!mkdir -p {data_dir}
!unzip -o {zip_name} -d {data_dir}

# Listar arquivos e abrir exemplo
base_path = "seed_pqd/XPQRS"

# Listar .csv
csv_files = sorted(glob.glob(os.path.join(base_path, "*.csv")))
print("Arquivos encontrados:")
for f in csv_files:
    print(" -", os.path.basename(f))

# Carregar Sag.csv
sag_path = os.path.join(base_path, "Sag.csv")
df_sag = pd.read_csv(sag_path, header=None)

print("\nFormato do Sag.csv:")
print("shape =", df_sag.shape)
print(df_sag.head())

Archive:  archive.zip
  inflating: seed_pqd/XPQRS/5Kfs_1Cycle_50f_1000Sam_1A.mat  
  inflating: seed_pqd/XPQRS/Details.txt  
  inflating: seed_pqd/XPQRS/Flicker.csv  
  inflating: seed_pqd/XPQRS/Flicker_with_Sag.csv  
  inflating: seed_pqd/XPQRS/Flicker_with_Swell.csv  
  inflating: seed_pqd/XPQRS/Harmonics.csv  
  inflating: seed_pqd/XPQRS/Harmonics_with_Sag.csv  
  inflating: seed_pqd/XPQRS/Harmonics_with_Swell.csv  
  inflating: seed_pqd/XPQRS/Interruption.csv  
  inflating: seed_pqd/XPQRS/Notch.csv  
  inflating: seed_pqd/XPQRS/Oscillatory_Transient.csv  
  inflating: seed_pqd/XPQRS/Pure_Sinusoidal.csv  
  inflating: seed_pqd/XPQRS/Sag.csv  
  inflating: seed_pqd/XPQRS/Sag_with_Harmonics.csv  
  inflating: seed_pqd/XPQRS/Sag_with_Oscillatory_Transient.csv  
  inflating: seed_pqd/XPQRS/Swell.csv  
  inflating: seed_pqd/XPQRS/Swell_with_Harmonics.csv  
  inflating: seed_pqd/XPQRS/Swell_with_Oscillatory_Transient.csv  
  inflating: seed_pqd/XPQRS/Transient.csv  
Arquivos encontrados:


In [ ]:
def rms_por_janela(sinal, n_pontos_janela):
    # Garantir que o sinal seja 2D para indexação consistente
    if sinal.ndim == 1:
        sinal = sinal[:, np.newaxis]

    Nt, Nsinais = sinal.shape
    num_janelas = Nt // n_pontos_janela
    rms_values = np.zeros((num_janelas, Nsinais))

    for s_idx in range(Nsinais):
        current_signal = sinal[:, s_idx]
        for i in range(num_janelas):
            start = i * n_pontos_janela
            end = start + n_pontos_janela
            window = current_signal[start:end]
            if len(window) > 0:
                rms_values[i, s_idx] = np.sqrt(np.mean(window**2))
            else:
                rms_values[i, s_idx] = np.nan  # Ou 0, dependendo do comportamento desejado
    return rms_values

def thd_por_janela(sinal, fs, f0, n_pontos_janela):
    # Garantir que o sinal seja 2D para indexação consistente
    if sinal.ndim == 1:
        sinal = sinal[:, np.newaxis]

    Nt, Nsinais = sinal.shape
    num_janelas = Nt // n_pontos_janela
    thd_values = np.zeros((num_janelas, Nsinais))

    for s_idx in range(Nsinais):
        current_signal = sinal[:, s_idx]
        for i in range(num_janelas):
            start = i * n_pontos_janela
            end = start + n_pontos_janela
            window = current_signal[start:end]

            if len(window) == 0:
                thd_values[i, s_idx] = np.nan
                continue

            # Executar FFT
            N = len(window)
            if N == 0:
                thd_values[i, s_idx] = np.nan
                continue
            yf = scipy.fft.fft(window)
            xf = scipy.fft.fftfreq(N, 1 / fs)

            # Considerar apenas frequências positivas (e ignorar componente DC)
            pos_freq_mask = xf > 0
            xf_pos = xf[pos_freq_mask]
            yf_pos = yf[pos_freq_mask]

            if len(xf_pos) == 0:
                thd_values[i, s_idx] = np.nan
                continue

            # Encontrar o índice da componente fundamental nas frequências positivas
            # Verificar se f0 está dentro do intervalo de frequências da janela
            if f0 < xf_pos[0] or f0 > xf_pos[-1]:
                thd_values[i, s_idx] = np.nan  # f0 não detectável nesta janela
                continue

            idx_f0_pos = np.argmin(np.abs(xf_pos - f0))
            fund_amplitude = np.abs(yf_pos[idx_f0_pos])

            harmonic_amplitudes_sq = 0
            # Iterar pelos harmônicos possíveis
            max_harmonic_order = min(25, int(fs / (2 * f0)) - 1)  # Limitado por Nyquist e garantir h*f0 < Nyquist

            for h in range(2, max_harmonic_order + 1):  # Do 2º até max_harmonic_order
                target_freq = h * f0
                if target_freq >= fs / 2:  # Excede a frequência de Nyquist
                    break

                idx_h_pos = np.argmin(np.abs(xf_pos - target_freq))
                harmonic_amplitudes_sq += (np.abs(yf_pos[idx_h_pos])**2)

            if fund_amplitude > 1e-9:  # Evitar divisão por zero ou fundamental muito pequena
                thd = (np.sqrt(harmonic_amplitudes_sq) / fund_amplitude) * 100
                thd_values[i, s_idx] = thd
            else:
                thd_values[i, s_idx] = 0.0  # Se fundamental ausente/pequena, THD pode ser 0 ou NaN
    return thd_values

def carregar_dataset_seed():
    base_path = "seed_pqd/XPQRS"

    def load_csv(nome_arquivo):
        caminho = os.path.join(base_path, nome_arquivo)
        df = pd.read_csv(caminho, header=None)
        return df.values

    banco_normal = load_csv("Pure_Sinusoidal.csv")
    banco_sag = load_csv("Sag.csv")
    banco_swell = load_csv("Swell.csv")
    banco_harm = load_csv("Harmonics.csv")
    banco_trans = load_csv("Oscillatory_Transient.csv")

    print("Shapes carregados da SEED:")
    print("Pure_Sinusoidal:", banco_normal.shape)
    print("Sag:", banco_sag.shape)
    print("Swell:", banco_swell.shape)
    print("Harmonics:", banco_harm.shape)
    print("Oscillatory_Transient:", banco_trans.shape)

    Nsamples = banco_sag.shape[0]
    Nciclos = 10
    f0_seed = 50.0
    Nppc_seed = Nsamples // Nciclos
    fs_seed = f0_seed * Nppc_seed

    t = np.arange(Nsamples) / fs_seed

    dataset_seed = {
        'transitorio': banco_trans,
        'swell': banco_swell,
        'sag': banco_sag,
        'harmonico': banco_harm,
        'normal': banco_normal
    }

    return dataset_seed, t, fs_seed, f0_seed, Nppc_seed

In [ ]:
# Carregar dataset REAL da SEED-PQD
dataset, tempo, fs, f0, Nppc = carregar_dataset_seed()

Shapes carregados da SEED:
Pure_Sinusoidal: (1000, 100)
Sag: (1000, 100)
Swell: (1000, 100)
Harmonics: (1000, 100)
Oscillatory_Transient: (1000, 100)


In [ ]:
def extrair_caracteristicas(dataset, t, fs, f0, Nppc):
    """Extrai características RMS e THD de cada sinal"""

    caracteristicas = []
    rotulos = []

    for tipo, banco in dataset.items():
        print(f"Processando {tipo}...")

        for i in range(banco.shape[1]):
            sinal = banco[:, i]

            rms_values = rms_por_janela(sinal, Nppc)
            thd_values = thd_por_janela(sinal, fs, f0, Nppc)

            rms_mean = np.mean(rms_values)
            rms_std = np.std(rms_values)
            rms_max = np.max(rms_values)
            rms_min = np.min(rms_values)

            thd_mean = np.mean(thd_values)
            thd_std = np.std(thd_values)
            thd_max = np.max(thd_values)

            amplitude_max = np.max(np.abs(sinal))
            variancia = np.var(sinal)

            feat_vector = [
                rms_mean, rms_std, rms_max, rms_min,
                thd_mean, thd_std, thd_max,
                amplitude_max, variancia
            ]

            caracteristicas.append(feat_vector)
            rotulos.append(tipo)

    return np.array(caracteristicas), np.array(rotulos)

# Extrair características
X, y = extrair_caracteristicas(dataset, tempo, fs, f0, Nppc)

print(f"Shape das características: {X.shape}")
print(f"Shape dos rótulos: {y.shape}")
print(f"Distribuição dos rótulos: {np.unique(y, return_counts=True)}")

Processando transitorio...
Processando swell...
Processando sag...
Processando harmonico...
Processando normal...
Shape das características: (500, 9)
Shape dos rótulos: (500,)
Distribuição dos rótulos: (array(['harmonico', 'normal', 'sag', 'swell', 'transitorio'], dtype='<U11'), array([100, 100, 100, 100, 100]))


In [ ]:
# Codificar rótulos
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print("Classes encontradas:", label_encoder.classes_)
print("Rótulos codificados únicos:", np.unique(y_encoded))

# Normalizar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\nPARÂMETROS DO STANDARD SCALER")
print("mean_ =", scaler.mean_)
print("scale_ =", scaler.scale_)

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

print(f"Treino: {X_train.shape}")
print(f"Teste:  {X_test.shape}")

Classes encontradas: ['harmonico' 'normal' 'sag' 'swell' 'transitorio']
Rótulos codificados únicos: [0 1 2 3 4]

PARÂMETROS DO STANDARD SCALER
mean_ = [6.76182727e-01 1.75993061e-02 7.05323873e-01 6.46414895e-01
 7.31438254e+02 7.11403631e+02 2.62744471e+03 1.07024088e+00
 2.22298690e-01]
scale_ = [2.77409474e-01 1.23819751e-02 2.88339510e-01 2.69158981e-01
 5.71951315e+02 1.13612935e+03 3.88537399e+03 5.01305976e-01
 2.45956382e-01]
Treino: (400, 9)
Teste:  (100, 9)


In [ ]:
import numpy as np

# Conferir mapeamento rótulo -> índice
print("Classes do LabelEncoder:", label_encoder.classes_)

# Calcular um vetor representativo (média) de cada classe em ESCALA ORIGINAL
vetores_por_classe = {}

for classe_nome in label_encoder.classes_:
    idx_classe = label_encoder.transform([classe_nome])[0]   # Ex: "harmonico" -> 0
    mask = (y_encoded == idx_classe)

    X_scaled_classe = X_scaled[mask]           # Todos os exemplos dessa classe (normalizados)
    mean_scaled = X_scaled_classe.mean(axis=0) # Média no espaço normalizado

    # Voltar para a escala original das features
    mean_original = scaler.inverse_transform(mean_scaled.reshape(1, -1))[0]

    vetores_por_classe[classe_nome] = mean_original

    print(f"\nClasse: {classe_nome}")
    for i, val in enumerate(mean_original):
        print(f"  feat[{i}] = {val:.4f}")

Classes do LabelEncoder: ['harmonico' 'normal' 'sag' 'swell' 'transitorio']

Classe: harmonico
  feat[0] = 0.7179
  feat[1] = 0.0272
  feat[2] = 0.7618
  feat[3] = 0.6671
  feat[4] = 921.3807
  feat[5] = 956.8025
  feat[6] = 3469.2947
  feat[7] = 1.3126
  feat[8] = 0.5136

Classe: normal
  feat[0] = 0.6364
  feat[1] = 0.0000
  feat[2] = 0.6364
  feat[3] = 0.6364
  feat[4] = 0.0000
  feat[5] = 0.0000
  feat[6] = 0.0000
  feat[7] = 0.6364
  feat[8] = 0.0000

Classe: sag
  feat[0] = 0.4812
  feat[1] = 0.0131
  feat[2] = 0.5029
  feat[3] = 0.4596
  feat[4] = 841.4310
  feat[5] = 744.9097
  feat[6] = 2790.9543
  feat[7] = 0.6364
  feat[8] = 0.0377

Classe: swell
  feat[0] = 0.8171
  feat[1] = 0.0199
  feat[2] = 0.8505
  feat[3] = 0.7871
  feat[4] = 876.7292
  feat[5] = 707.7319
  feat[6] = 2717.1852
  feat[7] = 1.1451
  feat[8] = 0.0293

Classe: transitorio
  feat[0] = 0.7282
  feat[1] = 0.0278
  feat[2] = 0.7750
  feat[3] = 0.6818
  feat[4] = 1017.6503
  feat[5] = 1147.5741
  feat[6] = 415

In [ ]:
# Treinar Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)

rf_model.fit(X_train, y_train)

# Avaliar
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Acurácia Random Forest: {accuracy_rf:.4f}")
print("\nRelatório de Classificação - Random Forest:")
print(classification_report(y_test, y_pred_rf, target_names=label_encoder.classes_))

Acurácia Random Forest: 0.9300

Relatório de Classificação - Random Forest:
              precision    recall  f1-score   support

   harmonico       1.00      1.00      1.00        20
      normal       0.91      1.00      0.95        20
         sag       0.82      0.90      0.86        20
       swell       0.94      0.75      0.83        20
 transitorio       1.00      1.00      1.00        20

    accuracy                           0.93       100
   macro avg       0.93      0.93      0.93       100
weighted avg       0.93      0.93      0.93       100



In [ ]:
def criar_modelo_mlp(input_dim, num_classes):
    model = keras.Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(num_classes, activation='linear')
    ])

    model.compile(
        optimizer='adam',
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    return model

# Criar e treinar o modelo
num_classes = len(label_encoder.classes_)
mlp_model = criar_modelo_mlp(X_train.shape[1], num_classes)

print(mlp_model.summary())

# Treinar
history = mlp_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Avaliar
test_loss, test_accuracy = mlp_model.evaluate(X_test, y_test, verbose=0)
print(f"\nAcurácia MLP: {test_accuracy:.4f}")

# Fazer previsões
y_pred_mlp = np.argmax(mlp_model.predict(X_test), axis=1)
print("\nRelatório de Classificação - MLP:")
print(classification_report(y_test, y_pred_mlp, target_names=label_encoder.classes_))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,885 (11.27 KB)

 Trainable params: 2,885 (11.27 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.1926 - loss: 1.6501 - val_accuracy: 0.4375 - val_loss: 1.4702
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3139 - loss: 1.5031 - val_accuracy: 0.6625 - val_loss: 1.3356
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4270 - loss: 1.3865 - val_accuracy: 0.7375 - val_loss: 1.2146
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4378 - loss: 1.2869 - val_accuracy: 0.7125 - val_loss: 1.1066
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5872 - loss: 1.1446 - val_accuracy: 0.7375 - val_loss: 1.0029
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5945 - loss: 1.0894 - val_accuracy: 0.6875 - val_loss: 0.9143
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6505 - loss: 0.9753 - val_accuracy: 0.6875 - val_loss: 0.8315
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6453 - loss: 0.9132 - val_accurac

In [ ]:
import matplotlib.pyplot as plt

# Plotar evolução do treinamento
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Loss do Modelo')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Matriz de confusão
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred_mlp)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.title('Matriz de Confusão - MLP')
plt.ylabel('Verdadeiro')
plt.xlabel('Predito')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

def visualizar_resultados_comparativos(y_test, y_pred_rf, y_pred_mlp, label_encoder, history=None):
    """Visualiza resultados comparativos para Random Forest e MLP"""

    # Métricas de comparação
    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

    print("=" * 60)
    print("COMPARAÇÃO ENTRE MODELOS")
    print("=" * 60)
    print(f"Random Forest Accuracy: {accuracy_rf:.4f}")
    print(f"MLP Accuracy: {accuracy_mlp:.4f}")
    print(f"Diferença: {abs(accuracy_rf - accuracy_mlp):.4f}")

    # Relatórios de classificação detalhados
    print("\n" + "=" * 40)
    print("RANDOM FOREST - Relatório Detalhado")
    print("=" * 40)
    print(classification_report(y_test, y_pred_rf, target_names=label_encoder.classes_))

    print("\n" + "=" * 40)
    print("MLP - Relatório Detalhado")
    print("=" * 40)
    print(classification_report(y_test, y_pred_mlp, target_names=label_encoder.classes_))

    # Gráficos comparativos
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # Matriz de Confusão: Random Forest
    cm_rf = confusion_matrix(y_test, y_pred_rf)
    sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', ax=axes[0,0],
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    axes[0,0].set_title('Matriz de Confusão - Random Forest')
    axes[0,0].set_ylabel('Verdadeiro')
    axes[0,0].set_xlabel('Predito')

    # Matriz de Confusão: MLP
    cm_mlp = confusion_matrix(y_test, y_pred_mlp)
    sns.heatmap(cm_mlp, annot=True, fmt='d', cmap='Greens', ax=axes[0,1],
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_)
    axes[0,1].set_title('Matriz de Confusão - MLP')
    axes[0,1].set_ylabel('Verdadeiro')
    axes[0,1].set_xlabel('Predito')

    # Comparação de Acurácia por Classe
    classes = label_encoder.classes_
    acc_rf_per_class = []
    acc_mlp_per_class = []

    for i, class_name in enumerate(classes):
        mask = y_test == i
        if np.sum(mask) > 0:
            acc_rf = np.mean(y_pred_rf[mask] == y_test[mask])
            acc_mlp = np.mean(y_pred_mlp[mask] == y_test[mask])
            acc_rf_per_class.append(acc_rf)
            acc_mlp_per_class.append(acc_mlp)

    x = np.arange(len(classes))
    width = 0.35

    axes[0,2].bar(x - width/2, acc_rf_per_class, width, label='Random Forest', alpha=0.7)
    axes[0,2].bar(x + width/2, acc_mlp_per_class, width, label='MLP', alpha=0.7)
    axes[0,2].set_title('Acurácia por Classe')
    axes[0,2].set_xlabel('Classe')
    axes[0,2].set_ylabel('Acurácia')
    axes[0,2].set_xticks(x)
    axes[0,2].set_xticklabels(classes, rotation=45)
    axes[0,2].legend()
    axes[0,2].grid(True, alpha=0.3)

    # Comparação de Acurácia Global
    models = ['Random Forest', 'MLP']
    accuracies = [accuracy_rf, accuracy_mlp]

    bars = axes[1,0].bar(models, accuracies, color=['blue', 'green'], alpha=0.7)
    axes[1,0].set_title('Acurácia Global dos Modelos')
    axes[1,0].set_ylabel('Acurácia')
    axes[1,0].set_ylim(0, 1)

    # Adicionar valores nas barras
    for bar, acc in zip(bars, accuracies):
        height = bar.get_height()
        axes[1,0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{acc:.4f}', ha='center', va='bottom')

    # Histórico de Treinamento do MLP
    if history is not None:
        axes[1,1].plot(history.history['accuracy'], label='Treino MLP')
        axes[1,1].plot(history.history['val_accuracy'], label='Validação MLP')
        axes[1,1].set_title('Evolução da Acurácia - MLP')
        axes[1,1].set_xlabel('Época')
        axes[1,1].set_ylabel('Acurácia')
        axes[1,1].legend()
        axes[1,1].grid(True, alpha=0.3)

        axes[1,2].plot(history.history['loss'], label='Treino MLP')
        axes[1,2].plot(history.history['val_loss'], label='Validação MLP')
        axes[1,2].set_title('Evolução da Loss - MLP')
        axes[1,2].set_xlabel('Época')
        axes[1,2].set_ylabel('Loss')
        axes[1,2].legend()
        axes[1,2].grid(True, alpha=0.3)
    else:
        # Mesmo comportamento do código antigo: importância das features do RF
        try:
            feature_importance = rf_model.feature_importances_
            feature_names = ['RMS_mean', 'RMS_std', 'RMS_max', 'RMS_min',
                             'THD_mean', 'THD_std', 'THD_max',
                             'Amp_max', 'Variancia']

            sorted_idx = np.argsort(feature_importance)
            axes[1,1].barh(range(len(sorted_idx)), feature_importance[sorted_idx])
            axes[1,1].set_yticks(range(len(sorted_idx)))
            axes[1,1].set_yticklabels([feature_names[i] for i in sorted_idx])
            axes[1,1].set_title('Importância das Features - Random Forest')
            axes[1,1].set_xlabel('Importância')

            axes[1,2].axis('off')
        except:
            axes[1,1].axis('off')
            axes[1,2].axis('off')

    plt.tight_layout()
    plt.show()

    # Variância e desvio-padrão dos ERROS (amostra a amostra)
    erro_rf  = (y_pred_rf  != y_test).astype(int)
    erro_mlp = (y_pred_mlp != y_test).astype(int)

    var_rf  = np.var(erro_rf)
    var_mlp = np.var(erro_mlp)
    std_rf  = np.std(erro_rf)
    std_mlp = np.std(erro_mlp)

    print("\n" + "=" * 60)
    print("VARIÂNCIA E DESVIO-PADRÃO DOS ERROS (amostra a amostra)")
    print("=" * 60)
    print(f"Random Forest  -> Var(erro): {var_rf:.4e} | Std(erro): {std_rf:.4e}")
    print(f"MLP            -> Var(erro): {var_mlp:.4e} | Std(erro): {std_mlp:.4e}")

    fig2, axes2 = plt.subplots(1, 2, figsize=(12, 5))

    axes2[0].bar(['RF', 'MLP'], [var_rf, var_mlp], color=['green', 'purple'])
    axes2[0].set_title('Variância dos Erros de Classificação')
    axes2[0].set_ylabel('Variância')
    axes2[0].grid(True, alpha=0.3)

    axes2[1].bar(['RF', 'MLP'], [std_rf, std_mlp], color=['green', 'purple'])
    axes2[1].set_title('Desvio-Padrão dos Erros de Classificação')
    axes2[1].set_ylabel('Desvio-Padrão')
    axes2[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return accuracy_rf, accuracy_mlp

print("Gerando visualizações comparativas...")
acc_rf, acc_mlp = visualizar_resultados_comparativos(
    y_test, y_pred_rf, y_pred_mlp, label_encoder, history
)

print("\n" + "=" * 50)
print("DECISÃO PARA IMPLEMENTAÇÃO NO ESP32")
print("=" * 50)

if acc_rf >= acc_mlp:
    print("RECOMENDAÇÃO: Usar Random Forest no ESP32")
    print("Motivos:")
    print("- Melhor acurácia")
    print("- Modelo mais leve e rápido")
    print("- Mais fácil de implementar em C puro")
else:
    print("RECOMENDAÇÃO: Usar MLP no ESP32")
    print("Motivos:")
    print("- Melhor acurácia")
    print("- Boa compatibilidade com TensorFlow Lite")

print(f"\nAcurácia Random Forest: {acc_rf:.4f}")
print(f"Acurácia MLP: {acc_mlp:.4f}")

In [ ]:
# Converter modelo para TensorFlow Lite (float32)
def converter_para_tflite(model):
    """
    Converte o modelo MLP para TensorFlow Lite em float32
    (sem quantização INT8: totalmente compatível com o ESP32)
    """
    converter = tf.lite.TFLiteConverter.from_keras_model(model)

    # Nenhuma quantização, nenhum representative_dataset
    # O modelo permanece em float32
    tflite_model = converter.convert()

    # Salvar o modelo em arquivo
    with open('modelo_disturbios_seed.tflite', 'wb') as f:
        f.write(tflite_model)

    print(f"Modelo TFLite float32 salvo como 'modelo_disturbios.tflite' ({len(tflite_model)} bytes)")
    return tflite_model


# Gerar modelo TFLite
tflite_model = converter_para_tflite(mlp_model)

print(f"Modelo TFLite criado! Tamanho: {len(tflite_model)} bytes")

# Testar o modelo TFLite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("\nDetalhes do modelo TFLite:")
print(f"Input:  {input_details[0]}")
print(f"Output: {output_details[0]}")

In [ ]:
# Gerar arquivo .h para o modelo TFLite float32
def gerar_codigo_arduino_corrigido(tflite_model, nome_array='modelo_disturbios'):
    """Gera um arquivo .h contendo o modelo TFLite (float32) para uso no ESP32"""

    hex_array = [f'0x{byte:02x}' for byte in tflite_model]

    c_code = f'''#ifndef {nome_array.upper()}_H
#define {nome_array.upper()}_H

// Modelo de IA para Qualidade de Energia - TensorFlow Lite (float32)
// Classes: [harmonico, normal, sag, swell, transitorio]
// Input: 9 features normalizadas pelo StandardScaler
// Output: 5 logits (camada densa linear)

#include <cstdint>

alignas(8) const unsigned char {nome_array}[] = {{
'''

    # Agrupar a saída em linhas de 12 bytes
    for i in range(0, len(hex_array), 12):
        line = ', '.join(hex_array[i:i+12])
        c_code += f'  {line},\n'

    c_code += f''' }};

const int {nome_array}_len = {len(tflite_model)};

#endif
'''

    # Salvar arquivo .h com o nome correto
    with open('modelo_disturbios_seed.h', 'w') as f:
        f.write(c_code)

    print("Arquivo 'modelo_disturbios_seed.h' gerado para Arduino!")
    print("Informações do modelo:")
    print(f"- Tamanho: {len(tflite_model)} bytes")

    return c_code

# Gerar arquivo .h
codigo_c = gerar_codigo_arduino_corrigido(tflite_model)

In [ ]:
from google.colab import files

# Listar arquivos no diretório atual
import os
print("Arquivos no diretório:")
for file in os.listdir('.'):
    if file.endswith('.h'):
        print(f"{file}")

# Download direto do arquivo
files.download('modelo_disturbios_seed.h')

# Visualização do conteúdo
with open('modelo_disturbios_seed.h', 'r') as f:
    conteudo = f.read()
    print("Primeiras 500 caracteres do arquivo:")
    print(conteudo[:500] + "...")

In [ ]:
# Exportar e baixar modelo .tflite

from google.colab import files
import tensorflow as tf

def exportar_tflite(modelo, nome='modelo_disturbios_seed.tflite'):
    converter = tf.lite.TFLiteConverter.from_keras_model(modelo)
    tflite_model = converter.convert()

    # Salvar no colab
    with open(nome, 'wb') as f:
        f.write(tflite_model)

    print(f'Arquivo salvo: {nome} ({len(tflite_model)} bytes)')

    # Baixar para o computador
    files.download(nome)
    return tflite_model


tflite_model = exportar_tflite(mlp_model)